In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='key-vault-secret')]

In [0]:
# Define storage account details
storage_account_name = "employeestorageproject"
access_key = dbutils.secrets.get(scope="key-vault-secret", key="storage-access-key")
client_id = dbutils.secrets.get(scope="key-vault-secret", key="client-id")
tenant_id = dbutils.secrets.get(scope="key-vault-secret", key="tenant-id")
client_secret = dbutils.secrets.get(scope="key-vault-secret", key="client-secret")


In [0]:
# Set the configuration
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}


In [0]:
container_names = ['raw', 'silver', 'gold']

In [0]:
for container_name in container_names:
    # Define the mount point
    mount_point = f"/mnt/{container_name}"

    # Unmount if already mounted (optional)
    try:
        if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
            dbutils.fs.unmount(mount_point)
    except:
        print(f"Mount point {mount_point} not found, proceeding")

    # Mount the storage
    dbutils.fs.mount(
        source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point=mount_point,
        extra_configs=configs
    )


    print(f"Mounted {container_name} to {mount_point}")


Mounted raw to /mnt/raw
/mnt/silver has been unmounted.
Mounted silver to /mnt/silver
/mnt/gold has been unmounted.
Mounted gold to /mnt/gold


In [0]:
display(dbutils.fs.mounts())


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/gold,abfss://gold@employeestorageproject.dfs.core.windows.net/,
/Volumes,UnityCatalogVolumes,
/mnt/silver,abfss://silver@employeestorageproject.dfs.core.windows.net/,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/raw,abfss://raw@employeestorageproject.dfs.core.windows.net/,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
